IMPORTING THE LIBRARIES

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest',
                                  validation_split=0.05)

IMPORTING THE DATASET

In [3]:
training_set = train_datagen.flow_from_directory('MaskData',
                                                target_size=(224,224),
                                                batch_size=16,
                                                class_mode='binary',
                                                subset = 'training')

Found 1783 images belonging to 2 classes.


In [9]:
validation_set = train_datagen.flow_from_directory('MaskData',
                                                       target_size=(224,224),
                                                       batch_size=16,
                                                       class_mode='binary',
                                                       subset = 'validation')

Found 93 images belonging to 2 classes.


TRAINING OF THE MODEL

In [10]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.models import Sequential
import numpy as np

vgg = VGG16(input_shape=[224,224]+[3],weights='imagenet', include_top=False)

In [11]:
for layer in vgg.layers:
    layer.trainable = False
    
x = Flatten()(vgg.output)
prediction = Dense(units=1,activation='sigmoid')(x)
model = Model(inputs= vgg.input,outputs=prediction)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [12]:
n = model.fit_generator(training_set,validation_data=validation_set,epochs=25,steps_per_epoch=len(training_set), validation_steps=len(validation_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
112/112 [==============================] - 622s 6s/step - loss: 0.4017 - accuracy: 0.8082 - val_loss: 0.5953 - val_accuracy: 0.6882
Epoch 2/25
112/112 [==============================] - 529s 5s/step - loss: 0.2586 - accuracy: 0.8884 - val_loss: 0.5082 - val_accuracy: 0.7849
Epoch 3/25
112/112 [==============================] - 498s 4s/step - loss: 0.2306 - accuracy: 0.9007 - val_loss: 0.4492 - val_accuracy: 0.8065
Epoch 4/25
112/112 [==============================] - 553s 5s/step - loss: 0.2077 - accuracy: 0.9136 - val_loss: 0.6851 - val_accuracy: 0.7204
Epoch 5/25
112/112 [==============================] - 554s 5s/step - loss: 0.2021 - accuracy: 0.9159 - val_loss: 0.4370 - val_accuracy: 0.7527
Epoch 6/25
112/112 [==============================] - 549s 5s/step - loss: 0.2282 - accuracy: 0.9080 - val_loss: 0.8890 - val_accuracy: 0.7742
Epoch 7/25
112/112 [==============================] - 428s 4s/step

In [13]:
model.save('mask_detector.h5')

In [4]:
print(training_set.class_indices)

{'masked': 0, 'unmask': 1}


In [ ]:
import cv2
import keras
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

model = keras.models.load_model('mask_detector.h5')

cap = cv2.VideoCapture(0)

while(1):
    _,img = cap.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    faces = face_cascade.detectMultiScale(gray,1.1,4)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h), (0,0,255),3)
        
        roi = img[y:y+h,x:x+w]
        
        image = cv2.resize(roi,(224,224))
        image2 = np.expand_dims(image,axis=0)   
        
        result = model.predict(image2)
        
        prediction = ''
        flag = 0
        
        if(result[0][0] == 0):
            cv2.rectangle(img,(x,y),(x+w,y+h), (255,0,0),3)
            prediction = 'MASK '
            flag = 1
        else:
            prediction = 'UNMASK'
            
        if flag == 0:
            font = cv2.FONT_HERSHEY_SIMPLEX
            img = cv2.putText(img, prediction, (x,y-5), font, 1, (0,0,255), 3, cv2.LINE_AA)
            
        else:
            font = cv2.FONT_HERSHEY_SIMPLEX
            img = cv2.putText(img, prediction, (x,y-5), font, 1, (255,0,0), 3, cv2.LINE_AA)
        
    cv2.imshow("RESULT",img)
    
cap.release()
cv2.destroyAllWindows()
        
    